# Linear Models
Timothy Helton

---
<br>
<font color="red">
    NOTE:
    <br>
    This notebook uses code found in the
    <a href="https://github.com/TimothyHelton/k2datascience/blob/master/k2datascience/regression.py">
    <strong>k2datascience.pca</strong></a> module.
    To execute all the cells do one of the following items:
    <ul>
        <li>Install the k2datascience package to the active Python interpreter.</li>
        <li>Add k2datascience/k2datascience to the PYTHON_PATH system variable.</li>
        <li>
            Create a link to the regression.py file in the same directory as this notebook.
        </li>
</font>

---
### Imports

In [ ]:
import os
import os.path as osp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import influence_plot
from statsmodels.sandbox.regression.predstd import wls_prediction_std

from k2datascience.utils import ax_formatter, save_fig, size

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

---
### Load Data

In [ ]:
data_dir = osp.realpath(osp.join(os.getcwd(), '..', 'data',
                                 'linear_regression'))

#### Batting

In [ ]:
batting = pd.read_csv(osp.join(data_dir, 'batting.csv'))
category_cols = (
    'stint',
    'league_id',
    'triple',
    'cs',
    'ibb',
    'hbp',
    'sf',
    'g_idp',
)
for col in category_cols:
    batting.loc[:, col] = batting.loc[:, col].astype('category')

batting.info()
batting.head()
batting.describe()

#### Player

In [ ]:
player = pd.read_csv(osp.join(data_dir, 'player.csv'))
category_cols = (
    'bats',
    'birth_month',
    'death_month',
    'throws',
)
for col in category_cols:
    player.loc[:, col] = player.loc[:, col].astype('category')

player.info()
player.head()
player.describe()

In [ ]:
salary = pd.read_csv(osp.join(data_dir, 'salary.csv'))
category_cols = (
    'team_id',
    'league_id',
)
for col in category_cols:
    salary.loc[:, col] = salary.loc[:, col].astype('category')

salary.info()
salary.head()
salary.describe()

In [ ]:
team = pd.read_csv(osp.join(data_dir, 'team.csv'))
category_cols = (
    'league_id',
    'div_id',
    'div_win',
    'lg_win',
    'rank',
    'team_id',
    'wc_win',
    'ws_win',
)
for col in category_cols:
    team.loc[:, col] = team.loc[:, col].astype('category')

team.info()
team.head()
team.describe()

## Exercise 1: 
1. Compute the correlation between mean salary and year.
1. Generate a graph of mean salary per year.

In [ ]:
mean_salary = (salary
               .groupby('year')
               .mean()
               .reset_index())
mean_salary.corr()

In [ ]:
ax = mean_salary.plot(x='year', y='salary', figsize=(8, 6),
                      label='Mean Salary')

ax.set_title('Mean Salary vs Year', fontsize=size['title'])
ax.legend(fontsize=size['legend'])
ax.set_xlabel('Year', fontsize=size['label'])
ax.set_ylabel('Mean Salary (x $1000)', fontsize= size['label'])
ax.yaxis.set_major_formatter(ax_formatter['thousands'])

plt.show();

## Exercise 2: 
1. Find the best line that approximates mean salary with respect to years. 
1. Plot this line together with the data from Exercise 1.

In [ ]:
lr = smf.ols(formula=f'salary ~ year', data=mean_salary).fit()
lr.summary()

In [ ]:
# Data
ax = mean_salary.plot(x='year', y='salary',
                      figsize=(8, 6), label='Mean Salary')

# Regression Line
ax.plot(mean_salary.year, lr.predict(mean_salary.year),
        linestyle='--', label='Linear Regression')

# Confidence Intervals
std, upper, lower = wls_prediction_std(lr)
ax.plot(mean_salary.year, lower, alpha=0.5, color='black',
        label='Confidence Interval', linestyle='-.')
ax.plot(mean_salary.year, upper, alpha=0.5, color='black',
        linestyle='-.')

ax.set_title('Mean Salary vs Year', fontsize=size['title'])
ax.legend(fontsize=size['legend'])
ax.set_xlabel('Year', fontsize=size['label'])
ax.set_ylabel('Mean Salary (x $1000)', fontsize= size['label'])
ax.yaxis.set_major_formatter(ax_formatter['thousands'])

plt.show();

## Exercise 3: Create a box plot for salaries per year.

In [ ]:
fig = plt.figure('Salary Boxp Plot', figsize=(12, 12),
                 facecolor='white', edgecolor='black')
rows, cols = (1, 1)
ax0 = plt.subplot2grid((rows, cols), (0, 0))

sns.boxplot(x='year', y='salary', data=salary,
            fliersize=2, ax=ax0)

ax0.set_title('Salary vs Year', fontsize=size['title'])
ax0.set_xlabel('Year', fontsize=size['label'])
ax0.set_ylabel('Salary (x $1000)', fontsize=size['label'])
ax0.yaxis.set_major_formatter(ax_formatter['thousands'])

fig.autofmt_xdate()

plt.show();

## Exercise 4:
1. From the previous graph we can see an increasing disparity in salaries as time increases. 
    1. How would you measure disparity in salaries? 
    1. Compute the correlation of disparity and years.
    1. Find the best line that approximates disparity with respect to years.

The [Gini Coefficient](https://en.wikipedia.org/wiki/Gini_coefficient) is a means to represent the income or wealth distribution of a population.
- The Gini coefficient measures the inequality among values of a frequency distribution.
- G = 0 represents perfect equality
- G = 1 expresses maximal inequality

$$G = \frac{2 \sum_{i=1}^n i y_i}{n \sum_{i=1}^n  y_i} - \frac{n + 1}{n}$$

In [ ]:
gini = {}
for year in salary.year.unique():
    salaries = (salary.query(f'year == {year}')
                .salary
                .sort_values())
    n = salaries.size

    gini[year] = ((2 * np.sum(salaries * (np.arange(n) + 1)))
                  / (n * salaries.sum())
                  - ((n + 1) / n))
gini = (pd.Series(gini)
        .reset_index()
        .rename(columns={'index': 'year', 0: 'gini'}))

In [ ]:
gini.corr()

In [ ]:
ax = gini.plot(x='year', y='gini', figsize=(8, 6),
               label='Gini Coefficient')

ax.set_title('Gini Coefficient vs Year', fontsize=size['title'])
ax.legend(fontsize=size['legend'])
ax.set_xlabel('Year', fontsize=size['label'])
ax.set_ylabel('Gini Coefficient', fontsize= size['label'])

plt.show();

In [ ]:
features = ' + '.join([f'np.power(year, {x + 1})' for x in range(2)])
quadratic_model = smf.ols(formula=f'gini ~ {features}', data=gini).fit()
quadratic_model.summary()

log_model = smf.ols(formula='gini ~ np.log(year) * year', data=gini).fit()
log_model.summary()

In [ ]:
fig = plt.figure('Regression Plot', figsize=(10, 5),
                 facecolor='white', edgecolor='black')
rows, cols = (1, 2)
ax0 = plt.subplot2grid((rows, cols), (0, 0))
ax1 = plt.subplot2grid((rows, cols), (0, 1), sharey=ax0)

# Regression Lines
ax0.plot(gini.year, quadratic_model.predict(gini.year),
         color='red', linestyle='-.',
         label='Quadratic Regression')
std0, upper, lower = wls_prediction_std(quadratic_model)
ax0.plot(gini.year, lower, alpha=0.5, color='black',
         label='Confidence Interval', linestyle='-.')
ax0.plot(gini.year, upper, alpha=0.5, color='black',
         linestyle='-.')

ax1.plot(gini.year, log_model.predict(gini.year),
         color='red', linestyle='--',
         label='Logrithmic Regression')
std, upper, lower = wls_prediction_std(log_model)
ax1.plot(gini.year, lower, alpha=0.5, color='black',
         label='Confidence Interval', linestyle='-.')
ax1.plot(gini.year, upper, alpha=0.5, color='black',
         linestyle='-.')

# Data
for ax in (ax0, ax1):
    gini.plot(x='year', y='gini', label='Gini Coefficient', ax=ax)
    ax.legend(fontsize=size['legend'])
    ax.set_xlabel('Year', fontsize=size['label'])
    ax.set_ylabel('Gini Coefficient', fontsize= size['label'])

fig.autofmt_xdate()
plt.tight_layout()
plt.suptitle('Gini Coefficient vs Year',
             fontsize=size['super_title'], y=1.05)

plt.show();

In [ ]:
fig = plt.figure('Residual Plot', figsize=(12, 5),
                 facecolor='white', edgecolor='black')
rows, cols = (1, 2)
ax0 = plt.subplot2grid((rows, cols), (0, 0))
ax1 = plt.subplot2grid((rows, cols), (0, 1))

# Quadratic Model
ax0.scatter(quadratic_model.fittedvalues, quadratic_model.resid)

ax0.set_title('Quadratic Model', fontsize=size['title'])

# Logrithmic Model
ax1.scatter(log_model.fittedvalues, log_model.resid)

ax1.set_title('Logrithmic Model', fontsize=size['title'])

for ax in (ax0, ax1):
    ax.set_xlabel('Fitted Values', fontsize=size['label'])
    ax.set_ylabel('Raw Residuals', fontsize=size['label'])
    
plt.show();

In [ ]:
fig = plt.figure('Influence Plot', figsize=(10, 10),
                 facecolor='white', edgecolor='black')
rows, cols = (1, 2)
ax0 = plt.subplot2grid((rows, cols), (0, 0))
ax1 = plt.subplot2grid((rows, cols), (0, 1))

# Quadradic Model
influence = influence_plot(quadratic_model, ax=ax0)

ax0.set_title('Quadratic Model',
              fontsize=size['title'])

# Logrithmic Model
influence = influence_plot(log_model, ax=ax1)

ax0.set_title('logrithmic Model',
              fontsize=size['title'])

for ax in (ax0, ax1):
    ax.set_xlabel('H Leverage',
                  fontsize=size['label'])
    ax.set_ylabel('Studentized Residuals',
                  fontsize=size['label'])

plt.show();

#### Findings
- Adding a cubic term yeilded similar results as the quadratic regression.
- Both models appear to be overfitting at the end of the data.
- The quadratic model does not fit the data as well as the logrithmic model.
- The quadratic model has a far smaller confidence interval than the logrithmic model.
- A small handful of data points are having a large impact on both models.

## Exercise 5: 
1. Build a predictive model for the amount of hits for a team given Games played, Wins, Walks by batters, At bats, Fielding  percentage, Outs Pitched (innings pitched x 3), Hits allowed, Earned runs allowed, Doubles. To solve this problem you will use team.csv. 
    1. How does your model measure accuracy?
    1. What was the score for its accuracy?
    1. Choose two features and create a 3d plot of feature1, feature2, h.

## Exercise 6:
1. Build a similar model to predict average hits per year based on Games played, at bats and whether a player is a left or right handed batter. Consider only those players who are either left or right handed batters and for the moment do not worry about missing data or ambidextrous batters. 